In [11]:
import csv
from scapy.all import *

In [4]:
packet_list = []
packet_list.append(['Src_MAC', 'Dest_MAC', 'Time', 'Len'])

In [7]:
def process_packet(packet):
    packet1 = []
    packet1.append(packet.src)
    packet1.append(packet.dst)
    packet1.append(packet.time)
    packet1.append(packet.len)
    packet_list.append(packet1)
#     print("SRC MAC: ", packet.src)
#     print("DST MAC: ", packet.dst)
#     print("ARRIVAL_TIME : ", packet.time)
#     print("LEN : ", packet.len)

In [9]:
sniff(offline="sample.pcap", prn=process_packet, store=0)
packet_list

[['Src_MAC', 'Dest_MAC', 'Time', 'Len'],
 ['60:32:b1:64:50:6d', 'ff:ff:ff:ff:ff:ff', Decimal('1646957193.773101'), 354],
 ['60:32:b1:64:50:6d', '01:00:5e:00:00:fb', Decimal('1646957198.281605'), 72],
 ['60:32:b1:64:50:6d', '01:00:5e:00:00:fb', Decimal('1646957198.292059'), 72],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.127740'), 408],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.127752'), 417],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128224'), 480],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128230'), 476],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128628'), 456],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129194'), 488],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129200'), 470],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129788'), 472],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129794'), 472],
 

In [12]:
with open('740_csv.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(packet_list)